## 📘 데이터 증강 후, 허깅페이스에 업로드해보기

- 본 프로젝트는 한국 뉴스 기사를 영어 지문으로 바꿔서 파인튜닝을 위한 데이터를 증강하는 프로젝트입니다.
- 영어 지문을 생성하는 모델을 파인튜닝해서 성능을 높여보고자 합니다.

### 필요한 라이브러리 설치

In [ ]:
# 필요한 라이브러리를 설치
!pip install -q openai==1.104.2
!pip install -q datasets==4.0.0

### 한국 뉴스 데이터 로드

In [ ]:
# 코랩 보안 비밀을 이용해 허깅페이스 로그인을 위한 키 가져오기
import huggingface_hub
from google.colab import userdata

huggingface_hub.login(userdata.get('huggingface_api_key'))

In [ ]:
from datasets import load_dataset
import pandas as pd

# Hugging Face 뉴스 데이터셋 로드
dataset_news = load_dataset("daekeun-ml/naver-news-summarization-ko")

# 데이터셋 중 'train'만 데이터프레임에 저장해보기
df = pd.DataFrame(dataset_news['train'])

In [ ]:
## 데이터 범주 간단하게 확인하기
df['category'].unique()

In [ ]:
import pandas as pd

# 테스트를 위함이므로, api 사용료를 고려하여 각 카테고리에서 50개씩 랜덤 샘플링해보기
df_sample = df.groupby('category', group_keys=False).apply(lambda x: x.sample(n=50, random_state=42))

# 결과 확인
print(df_sample['category'].value_counts())

### 파인튜닝을 위한 데이터 리스트화 및 생성

In [ ]:
# 기사 원문 리스트화 해보기
doc_content = df_sample['document'].to_list()
doc_content

In [ ]:
from pydantic import BaseModel
import openai
from google.colab import userdata

# 응답 형식 지정
class Passage(BaseModel):
    field : str
    topic : str
    passage : str

client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# 뉴스 기사를 바탕으로 영어 지문/분야/주제 생성해보기
def get_passages_from_news(news_content):
  response = client.beta.chat.completions.parse(
    model="gpt-4.1",
    messages=[
          {"role": "system", "content": f'You are responsible for creating an English passage based on the given Korean news article so that students can solve questions derived from it.\n'
                                      'Based on the news article, write an English passage, specify the field of the passage, and provide its topic.\n\n'
                                      '## Guidelines for Passage Writing\n'
                                      '- The passage must address fair and objective facts, excluding private or personal information such as specific company names.\n'
                                      '- Include appropriate examples or conceptual explanations, but do not describe fictional events, concepts, or imaginary figures.\n'
                                      '- Avoid unnecessary repetition of the same information or arguments; ensure logical progression of sentences.\n'
                                      '- The passage length must be between 800 and 1200 characters, including spaces.\n'
                                      '- Incorporate sufficient elements that encourage problem-solving.\n'
                                      '- All sentences must be written in English with perfect grammar. \n\n'
                                      '## Fields of the Passage\n'
                                      '- Choose one from: Humanities / Science / Technology / Society / Arts. \n\n'
                                      '## Topic of the Passage\n'
                                      '- Express it as a short phrase based on the core keywords.'},
          {"role": "user", "content": "The Korean news article is as follows.\n"
                                      f"'{news_content}'\n\n"
                                      "The answer format should be:\n\n"
                                      "{\n"
                                      " passage = str (the passage)\n"
                                      " subject = str (the field of the passage)\n"
                                      " topic = str (the topic of the passage)\n"
                                      "}"
                                      }
      ],
    temperature=0.2,
    response_format=Passage,
  )
  return response.choices[0].message.parsed

In [ ]:
# 데이터 생성 진행 상황 살펴보기
from tqdm import tqdm

news_passage_content = []
for a in tqdm(doc_content):  # zip을 사용하여 여러 리스트를 묶음
    news_passage_content.append(get_passages_from_news(a))

In [ ]:
# 생성된 지문 분야/주제/영어 지문 리스트 추출
field_gen = [item.field for item in news_passage_content]
topic_gen = [item.topic for item in news_passage_content]
passage_gen = [item.passage for item in news_passage_content]


# 기존 df_sample에 컬럼 추가
df_sample['field'] = field_gen
df_sample['topic'] = topic_gen
df_sample['assistant'] = passage_gen
df_sample

In [ ]:
## csv로 잘 저장되나 테스트해보기
df_sample.to_csv('news_passage_data.csv', index=False, encoding='utf-8-sig')
df_sample_test = pd.read_csv('news_passage_data.csv', encoding='utf-8-sig')
df_sample_test

In [ ]:
## 파인튜닝을 위한 시스템 프롬프트 만들기
system_prompt = ("You are a test item developer who generates English passages based on given Korean news articles.\n"
                 "You must create an English passage that is grounded in academic content, deals with fair and objective facts, and reflects the specified field and topic provided by the user.\n"
                 "The passage should maintain a coherent flow around a single theme,\n"
                 "without unnecessary repetition of the same information or similar arguments, while developing each sentence logically.\n"
                 "Rather than merely listing information, you should connect concepts organically and describe them logically.\n"
                 "The passage must be written in such a way that students can engage in logical reasoning while reading it.\n"
                 "All sentences must be generated in English and be grammatically flawless.")

In [ ]:
## 파인튜닝을 위한 유저 프롬프트 만들기
from tqdm import tqdm

def get_user_prompt(field, topic):
    messages=[
            {"role": "user", "content": f"Please create an English passage\n"
                                        f"Field : {field}\n"
                                        f"Topic : {topic}"
                                        }
    ]
    return messages[0]['content']

user_prompt = []
for _, row in tqdm(df_sample.iterrows()):
    user_prompt.append(get_user_prompt(row['field'], row['topic']))

In [ ]:
# 파인튜닝을 위한 데이터 추가
## 파인튜닝 데이터는 온전히 영어로 이루어짐으로써 사용자가 분야, 주제만 입력했을 때 고품질 지문을 받도록 하고자 함
df_sample['user_prompt'] = user_prompt
df_sample['system_prompt'] = system_prompt
df_sample

In [ ]:
# 데이터 1차 csv 저장 및 파인튜닝을 위한 데이터 확정
df_sample.to_csv('data_news_0907.csv', index=False, encoding='utf-8-sig')
df_final = df_sample[['system_prompt','user_prompt','assistant']].reset_index(drop=True)
df_final.columns = ['system', 'user', 'assistant']
df_final

In [ ]:
# json으로도 저장해보기
output_file = "fine_tuning_data_0907.json"

df_final.to_json(
    output_file, orient="records", force_ascii=False, indent=4
)

### 데이터 train/valid/test 분할 및 허깅페이스 업로드

In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 먼저 train(80%) / temp(20%)로 나누기
df_train, df_temp = train_test_split(
    df_final, test_size=0.2, random_state=42
)

# temp를 다시 validation(50%) / test(50%)로 나눠서 각각 10%씩 만들기
df_valid, df_test = train_test_split(
    df_temp, test_size=0.5, random_state=42
)

# pandas → Hugging Face Dataset 변환
dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "validation": Dataset.from_pandas(df_valid.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
})

In [ ]:
## 만약 openai에서 파인튜닝한다면 데이터를 다음의 양식에 맞춰서 만들어야 함
import json
import pandas as pd
import random

# JSONL 파일 저장 경로
output_train_file = "fine_tuning_train_data_0907.jsonl"
output_valid_file = "fine_tuning_valid_data_0907.jsonl"
output_test_file = "fine_tuning_test_data_0907.jsonl"

# 함수: JSONL 파일로 저장
def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in data.iterrows():
            entry = {
                "messages": [
                    {"role": "system", "content": row["system"]},
                    {"role": "user", "content": row["user"]},
                    {"role": "assistant", "content": row["assistant"]}
                ]
            }
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
    print(f"{filename} 저장 완료!")

# 파일로 저장
save_jsonl(df_train, output_train_file)
save_jsonl(df_valid, output_valid_file)
save_jsonl(df_test, output_test_file)

In [ ]:
# 허깅페이스 데이터 업로드
from datasets import Dataset, DatasetDict
from huggingface_hub import HfApi

dataset_dict.push_to_hub("kkobuking/finetuning_test_data_0907", private=False)

print("Dataset uploaded successfully!")

In [ ]:
import pandas as pd
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("kkobuking/finetuning_test_data_0907")

# train/validation/test 각각을 DataFrame으로 변환
df_train1 = dataset['train'].to_pandas()
df_valid1 = dataset['validation'].to_pandas()
df_test1 = dataset['test'].to_pandas()

# 하나의 DataFrame으로 합치기
df_all = pd.concat([df_train1, df_valid1, df_test1], ignore_index=True)

# 확인
df_all

In [ ]:
from huggingface_hub import delete_repo

# 데이터셋 삭제 (repo_id: "사용자이름/데이터셋이름")
delete_repo(repo_id="kkobuking/finetuning_test", repo_type="dataset")